# Required modules

In [1]:
# import system modules
import ee
import os
import sys
import random
import pandas as pd
from pprint import pprint

In [4]:
# import user modules
sys.dont_write_bytecode = True
sys.path.append(os.path.abspath('../'))

from modules.Collection import getCollection
from modules.BandNames import getBandNames
from modules.CloudAndShadowMaskC2 import getMasks
from modules.SMA_NDFI import *


In [32]:
pd.set_option('mode.chained_assignment', None)

In [33]:
# intialize earth engine credentials
ee.Initialize()

# Random Forest parameterization

In [34]:
# random forest params
RF_PARAMS = {
    'numberOfTrees': 50,
    # 'variablesPerSplit': 4,
    # 'minLeafPopulation': 25
}

# define the feature space
FEAT_SPACE_BANDS = [
    "gv",
    "gvs",
    "soil",
    "npv",
    "shade",
    "ndfi",
    "csfi",
]


# Sampling parameters

In [35]:
# samples version
V_SAMPLES = '1'

# full asset name of samples folder
ASSET_SAMPLES = 'projects/earthengine-legacy/assets/projects/imazon-simex/LULC/SAMPLES/COLLECTION7/TRAINED_BY_SCENE'

# number of samples
N_SAMPLES = 2000

# minimum number of samples per classe
dfMinSamples = pd.DataFrame([
    {'class':  3, 'min_samples': N_SAMPLES * 0.10},
    {'class':  4, 'min_samples': N_SAMPLES * 0.05},
    {'class': 12, 'min_samples': N_SAMPLES * 0.05},
    {'class': 15, 'min_samples': N_SAMPLES * 0.10},
    {'class': 18, 'min_samples': N_SAMPLES * 0.10},
    {'class': 25, 'min_samples': N_SAMPLES * 0.05},
    {'class': 33, 'min_samples': N_SAMPLES * 0.10},
])

dfMinSamples


,class,min_samples
0,3,200.0
1,4,100.0
2,12,100.0
3,15,200.0
4,18,200.0
5,25,100.0
6,33,200.0


# Landsat variables specification

In [36]:
# landsat short names
SATELLITE_IDS = [
    # 'l5',
    'l7',
    # 'l8',
    # 'l9'
    ]

# endemembers used to unmix data
ENDMEMBERS = {
    'l5': ENDMEMBERS_L5,
    'l7': ENDMEMBERS_L7,
    'l8': ENDMEMBERS_L8,
    'l9': ENDMEMBERS_L8,
}

# landsat collection ids
COLLECTION_IDS = {
    'l5': 'LANDSAT/LT05/C02/T1_L2',
    'l7': 'LANDSAT/LE07/C02/T1_L2',
    'l8': 'LANDSAT/LC08/C02/T1_L2',
    'l9': 'LANDSAT/LC09/C02/T1_L2',
}

# Ancillary data
## 1. Area per classe and landsat tile

In [37]:
# read areas table
dfAreas = pd.read_csv('../data/areas.csv')
dfAreas

,tile,year,class,area
0,1057.0,1985,3,6222.587536
1,1057.0,1985,12,0.368480
2,1057.0,1985,15,7.589520
3,1057.0,1985,33,40.358827
4,1058.0,1985,3,13088.503428
...,...,...,...,...
46404,233067.0,2022,33,249.164548
46405,233068.0,2022,3,5927.045667
46406,233068.0,2022,12,261.149815
46407,233068.0,2022,15,297.970598


## 2. Priority table

In [38]:
# # reads priority table
# dfPriority = pd.read_csv('../data/priority.csv', decimal=',')

# # find which tiles will be processed
# dfPriority['process'] = (dfPriority['P0'] > 0.9) & (dfPriority['P0'] < 0.97)

# # takes the relevant columns
# dfPriority = dfPriority[['tile', 'process']]

# dfPriority

## 3. Defective image list

In [39]:
# loads image list from trash
trash = pd.read_json('../data/trash.json')

trash = list(trash.image_id.values)

trash

['LT05_231068_19850823',
 'LT05_231068_19850908',
 'LT05_227067_19850827',
 'LT05_227067_19850912',
 'LT05_224065_19850822',
 'LT05_224065_19851025',
 'LT05_227067_19870817',
 'LT05_226068_19870911',
 'LT05_226068_19870826',
 'LT05_226068_19870810',
 'LT05_232068_19870921',
 'LT05_232068_19870905',
 'LT05_232068_19870820',
 'LT05_232068_19870804',
 'LT05_230069_19880925',
 'LT05_230069_19880909',
 'LT05_230069_19880824',
 'LT05_229069_19880902',
 'LT05_227063_19880819',
 'LT05_227063_19880904',
 'LT05_227063_19881006',
 'LT05_229069_19890820',
 'LT05_229069_19890921',
 'LT05_229068_19890921',
 'LT05_231060_19901008',
 'LT05_233060_19910822',
 'LT05_229069_19910927',
 'LT05_229069_19910911',
 'LT05_221063_19911106',
 'LT05_221063_19911021',
 'LT05_221063_19910919',
 'LT05_221063_19910903',
 'LT05_223067_19940925',
 'LT05_223067_19940808',
 'LT05_223067_19941027',
 'LT05_223066_19940925',
 'LT05_229071_19951008',
 'LT05_229071_19950906',
 'LT05_228071_19951001',
 'LT05_228071_19950830',


## 4. Landsat tiles collection

In [40]:
# loads ladsat tiles collection
ASSET_TILES = 'projects/mapbiomas-workspace/AUXILIAR/landsat-mask'

tilesCollection = ee.ImageCollection(ASSET_TILES)\
    .filterMetadata('version', 'equals', '2')

## 5. Output collection

In [41]:
# output asset path
ASSET_OUTPUT = 'projects/imazon-simex/LULC/COLLECTION7/classification'

# output files version
OUTPUT_VERSION = '4'

# loads files from output collection
outputCollection = ee.ImageCollection(ASSET_OUTPUT)\
    .filter(ee.Filter.eq('version', OUTPUT_VERSION))

# lists image files already saved in the collection
outputAssetIds = outputCollection.reduceColumns(
        ee.Reducer.toList(), ['system:index'])\
        .get('list')\
        .getInfo()

# list of landsat image ids to skip
skipList = map(
    lambda imageid: imageid.replace('_' + OUTPUT_VERSION, ''),
    outputAssetIds
)

skipList = list(skipList)

skipList

['LC08_001057_20210916',
 'LC08_001057_20211002',
 'LC08_001057_20220106',
 'LC08_001057_20220428',
 'LC08_001057_20220530',
 'LC08_001057_20220717',
 'LC08_001057_20220919',
 'LC08_001057_20221224',
 'LC08_001058_20210324',
 'LC08_001058_20210916',
 'LC08_001058_20211018',
 'LC08_001058_20220106',
 'LC08_001058_20220428',
 'LC08_001058_20220514',
 'LC08_001058_20220530',
 'LC08_001058_20220717',
 'LC08_001058_20220818',
 'LC08_001058_20220919',
 'LC08_001058_20221122',
 'LC08_001058_20221224',
 'LC08_001059_20210204',
 'LC08_001059_20210628',
 'LC08_001059_20210916',
 'LC08_001059_20211205',
 'LC08_001059_20220106',
 'LC08_001059_20220428',
 'LC08_001059_20220514',
 'LC08_001059_20220717',
 'LC08_001059_20220818',
 'LC08_001059_20220919',
 'LC08_001059_20221122',
 'LC08_001059_20221224',
 'LC08_001060_20210204',
 'LC08_001060_20210628',
 'LC08_001060_20210714',
 'LC08_001060_20210815',
 'LC08_001060_20210916',
 'LC08_001060_20211002',
 'LC08_001060_20211205',
 'LC08_001060_20220106',


# Preparing data table
## 1. Legend adjustment for Amazon classes

In [42]:
INDEX = ['year', 'tile']

dfAreas = dfAreas.replace({'class': {20: 18, 36: 18, 39: 18, 41: 18, 24: 25, 9:3}})

# aggregate areas by class, tile and year
dfAgg = dfAreas.groupby(['year', 'tile', 'class']).agg({'area': 'sum'}).reset_index()

# calculate the total area per tile and year
dfTotal = dfAreas.groupby(INDEX).agg({'area': 'sum'}).reset_index()

# merges the dfAgg with dfTotal
df = pd.merge(dfAgg, dfTotal, how="outer", on=INDEX, suffixes=(None, '_total'))

df[(df['tile'] == 226069) & (df['year'] == 2021)]

,year,tile,class,area,area_total
40442,2021,226069.0,3,12715.134341,26779.2821
40443,2021,226069.0,4,1365.140159,26779.2821
40444,2021,226069.0,12,42.752283,26779.2821
40445,2021,226069.0,15,1337.370450,26779.2821
40446,2021,226069.0,18,11228.152083,26779.2821
40447,2021,226069.0,25,33.303733,26779.2821
40448,2021,226069.0,33,57.429050,26779.2821


## 2. Calculate the proportion of each class

In [43]:
df['proportion'] = df['area'].div(df['area_total'])

df[(df['tile'] == 226069) & (df['year'] == 2021)]

,year,tile,class,area,area_total,proportion
40442,2021,226069.0,3,12715.134341,26779.2821,0.474812
40443,2021,226069.0,4,1365.140159,26779.2821,0.050977
40444,2021,226069.0,12,42.752283,26779.2821,0.001596
40445,2021,226069.0,15,1337.370450,26779.2821,0.049940
40446,2021,226069.0,18,11228.152083,26779.2821,0.419285
40447,2021,226069.0,25,33.303733,26779.2821,0.001244
40448,2021,226069.0,33,57.429050,26779.2821,0.002145


## 3. Number of samples based on proportions of area

In [44]:
# calculates the number of samples based on proportions
df['n_samples'] = df['proportion'].mul(N_SAMPLES).round()

df[(df['tile'] == 226069) & (df['year'] == 2021)]

,year,tile,class,area,area_total,proportion,n_samples
40442,2021,226069.0,3,12715.134341,26779.2821,0.474812,950.0
40443,2021,226069.0,4,1365.140159,26779.2821,0.050977,102.0
40444,2021,226069.0,12,42.752283,26779.2821,0.001596,3.0
40445,2021,226069.0,15,1337.370450,26779.2821,0.049940,100.0
40446,2021,226069.0,18,11228.152083,26779.2821,0.419285,839.0
40447,2021,226069.0,25,33.303733,26779.2821,0.001244,2.0
40448,2021,226069.0,33,57.429050,26779.2821,0.002145,4.0


## 4. Compare the `min_samples` to `n_samples` and keep the highest value

In [45]:
# merges minimum samples per class to data table
df = pd.merge(df, dfMinSamples, how="outer", on="class")

# replace n_samples column with the highest value betwen min_samples and n_samples
df.loc[df['min_samples'] > df['n_samples'], 'n_samples'] = df['min_samples']

df[(df['tile'] == 226069) & (df['year'] == 2021)]
    

,year,tile,class,area,area_total,proportion,n_samples,min_samples
7380,2021,226069.0,3,12715.134341,26779.2821,0.474812,950.0,200.0
14941,2021,226069.0,12,42.752283,26779.2821,0.001596,100.0,100.0
22547,2021,226069.0,15,1337.370450,26779.2821,0.049940,200.0,200.0
30174,2021,226069.0,33,57.429050,26779.2821,0.002145,200.0,200.0
35187,2021,226069.0,25,33.303733,26779.2821,0.001244,100.0,100.0
37131,2021,226069.0,4,1365.140159,26779.2821,0.050977,102.0,100.0
40504,2021,226069.0,18,11228.152083,26779.2821,0.419285,839.0,200.0


## 5. Number of regional and global samples

In [46]:
# df['rg_samples'] = df['n_samples'].mul(P_SAMPLES['regional']).round()
# df['gl_samples'] = df['n_samples'].mul(P_SAMPLES['global']).round()

# df[(df['tile'] == 226069) & (df['year'] == 2020)]

In [47]:
# merges minimum samples per class to data table
# df = pd.merge(df, dfPriority, how="outer", on="tile")

# df[(df['tile'] == 226069) & (df['year'] == 2020)]

# Utilities functions

In [48]:
def shuffle(collection, seed=1):
    """
    Adds a column of deterministic pseudorandom numbers to a collection.
    The range 0 (inclusive) to 1000000000 (exclusive).
    """

    collection = collection.randomColumn('random', seed)\
        .sort('random', True)\
        .map(
            lambda feature: feature.set(
                'new_id',
                ee.Number(feature.get('random'))
                .multiply(1000000000)
                .round()
            )
    )

    # list of random ids
    randomIdList = ee.List(
        collection.reduceColumns(ee.Reducer.toList(), ['new_id'])
        .get('list'))

    # list of sequential ids
    sequentialIdList = ee.List.sequence(1, collection.size())

    # set new ids
    shuffled = collection.remap(randomIdList, sequentialIdList, 'new_id')

    return shuffled

In [49]:
def applyCloudAndShadowMask(collection):

    # Get cloud and shadow masks
    collectionWithMasks = getMasks(collection,
                                   cloudFlag=True,
                                   cloudScore=True,
                                   cloudShadowFlag=True,
                                   cloudShadowTdom=True,
                                   zScoreThresh=-1,
                                   shadowSumThresh=4000,
                                   dilatePixels=2,
                                   cloudHeights=ee.List.sequence(
                                       200, 10000, 500),
                                   cloudBand='cloudFlagMask')

    # collectionWithMasks = collectionWithMasks.select(specCloudBands)

    # get collection without clouds
    collectionWithoutClouds = collectionWithMasks.map(
        lambda image: image.mask(
            image.select([
                'cloudFlagMask',
                'cloudScoreMask',
                'cloudShadowFlagMask',
                'cloudShadowTdomMask'
            ]).reduce(ee.Reducer.anyNonZero()).eq(0)
        )
    )

    return collectionWithoutClouds

In [50]:
def classify(image, samples, featSpace, numberOfTrees, variablesPerSplit=None, minLeafPopulation=None):

    classifier = ee.Classifier.smileRandomForest(
        numberOfTrees=numberOfTrees
    ).train(samples, 'class', featSpace)

    return ee.Image(image
                    .select(featSpace)
                    .classify(classifier)
                    .rename(['classification'])
                    .copyProperties(image)
                    .copyProperties(image, ['system:footprint'])
                    .copyProperties(image, ['system:time_start'])
                    )

In [51]:
def getSamples(folder, tiles=None, shuffled=True):

    assetList = ee.data.listAssets({'parent':folder})
    
    if tiles:
        assetList = filter(
            lambda obj: 
                int(obj['name'].split('/')[-1].split('_')[1]) in tiles,
                assetList['assets']
            )
    else:
        assetList = assetList['assets']
        
    samplesList = map(
        lambda obj: ee.FeatureCollection(obj['name']),
        list(assetList)
    )

    samples = ee.FeatureCollection(list(samplesList)).flatten()

    if shuffled:
        samples = shuffle(samples)

    return samples


# Iterate over years and landsat tiles

In [52]:
# years = list(pd.unique(df['year']))
years = [2022]

tiles = list(pd.unique(df['tile']))

print(tiles)


[1057.0, 1058.0, 1059.0, 1060.0, 1061.0, 1062.0, 1063.0, 1064.0, 1065.0, 1066.0, 1067.0, 2057.0, 2059.0, 2060.0, 2061.0, 2062.0, 2063.0, 2064.0, 2065.0, 2066.0, 2067.0, 2068.0, 3058.0, 3059.0, 3060.0, 3061.0, 3062.0, 3063.0, 3064.0, 3065.0, 3066.0, 3067.0, 3068.0, 4059.0, 4060.0, 4061.0, 4062.0, 4063.0, 4064.0, 4065.0, 4066.0, 4067.0, 5059.0, 5060.0, 5063.0, 5064.0, 5065.0, 5066.0, 5067.0, 6063.0, 6064.0, 6065.0, 6066.0, 220062.0, 220063.0, 221061.0, 221062.0, 221063.0, 221064.0, 222061.0, 222062.0, 222063.0, 222064.0, 222065.0, 222066.0, 223060.0, 223061.0, 223062.0, 223063.0, 223064.0, 223065.0, 223066.0, 223067.0, 224060.0, 224061.0, 224062.0, 224063.0, 224064.0, 224065.0, 224066.0, 224067.0, 224068.0, 224069.0, 225058.0, 225059.0, 225060.0, 225061.0, 225062.0, 225063.0, 225064.0, 225065.0, 225066.0, 225067.0, 225068.0, 225069.0, 225070.0, 226057.0, 226058.0, 226059.0, 226060.0, 226061.0, 226062.0, 226063.0, 226064.0, 226065.0, 226066.0, 226067.0, 226068.0, 226069.0, 227058.0, 22705

In [53]:
for year in years:

    for tile in tiles:

        tileMask = tilesCollection.filterMetadata('tile', 'equals', int(tile))
        tileMask = ee.Image(tileMask.first())
        
        geometry = tileMask.geometry()
        centroid = geometry.centroid()

        dfTileYear = df[(df['tile'] == tile) & (df['year'] == year)]

        for satelliteId in SATELLITE_IDS:
            try:
                # returns a collection containing the specified parameters
                collection = getCollection(COLLECTION_IDS[satelliteId],
                                            dateStart=str(year)+'-01-01',
                                            dateEnd=str(year)+'-12-31',
                                            cloudCover=50,
                                            geometry=centroid)
                
                # drops images in trash and skip lists
                collection = collection\
                    .filter(ee.Filter.inList('system:index', skipList).Not())\
                    .filter(ee.Filter.inList('system:index', trash).Not())
            
                # returns the pattern of band names
                bands = getBandNames(satelliteId + 'c2')

                # selects the images bands and rename it
                collection = collection.select(
                    bands['bandNames'],
                    bands['newNames']
                )

                # remove clouds and shadows
                collectionWithoutClouds = applyCloudAndShadowMask(collection)

                # build the feature space bands
                featureSpaceCollection = collectionWithoutClouds\
                    .map(lambda image:
                        image.addBands(srcImg=getSMAFractions(image, ENDMEMBERS[satelliteId]), overwrite=True))\
                    .map(lambda image: 
                        image.addBands(srcImg=getNDFI(image), overwrite=True))\
                    .map(lambda image: 
                        image.addBands(srcImg=getCSFI(image), overwrite=True))
                
                # lists the image ids 
                imageIds = collection.reduceColumns(
                    ee.Reducer.toList(), ['system:index'])

                imageIds = imageIds.get('list').getInfo()
                
                for imageId in imageIds:
                    
                    print(imageId, OUTPUT_VERSION)

                    # step 1: get the image
                    image = featureSpaceCollection.filterMetadata(
                        'system:index', 'equals', imageId)
                    
                    image = ee.Image(image.first())

                    # step 2: select the samples
                    samplesName = '{}/{}/{}_{}'.format(
                        ASSET_SAMPLES, year, imageId, V_SAMPLES)

                    samples = ee.FeatureCollection(samplesName)\
                        .filter(ee.Filter.neq('gv', None))
                    
                    # step 3: merges rgSamples with glSamples
                    samplesFolder = '{}/{}'.format(ASSET_SAMPLES, year)
                    
                    samplesSelectTiles = getSamples(samplesFolder,
                                                    tiles=[int(tile)],
                                                    shuffled=False)

                    # random sampling x% of tiles
                    randomTiles = random.sample(tiles, int(len(tiles) * 0.3))
                    randomTiles = list(map(
                        lambda randomTile: int(randomTile), randomTiles))
                    
                    samplesRandomTiles = getSamples(samplesFolder,
                                                    tiles=randomTiles,
                                                    shuffled=True)
                    allSamples = samples\
                        .merge(samplesSelectTiles)\
                        .merge(samplesRandomTiles)

                    # pprint(allSamples.aggregate_histogram('class').getInfo())

                    dfTileYear['samples_gee'] = dfTileYear.apply(
                        lambda serie: allSamples
                            .filter(ee.Filter.eq('class', serie['class']))
                            .limit(serie['n_samples']),
                        axis=1
                    )

                    trainingSamples = ee.FeatureCollection(
                        list(dfTileYear['samples_gee'].values)).flatten()

                    # step 4: image classification
                    classification = classify(
                        image=image,
                        samples=trainingSamples,
                        featSpace=FEAT_SPACE_BANDS,
                        numberOfTrees=RF_PARAMS['numberOfTrees']
                        )

                    classification = classification.updateMask(tileMask)
                    classification = classification.toByte()
                    classification = classification.set('version', OUTPUT_VERSION)
                    classification = classification.set('biome', 'AMAZONIA')
                    classification = classification.set('territory', 'AMAZONIA')
                    classification = classification.set('source', 'Imazon')
                    classification = classification.set('year', year)

                    # step 5: export classification to gee asset
                    imageName = '{}_{}'.format(imageId, OUTPUT_VERSION)
                    
                    assetId = '{}/{}'.format(ASSET_OUTPUT, imageName)
                    
                    region = geometry.getInfo()['coordinates']

                    task = ee.batch.Export.image.toAsset(
                        image=classification,
                        description=imageName,
                        assetId=assetId,
                        pyramidingPolicy={".default": "mode"},
                        region=region,
                        scale=30,
                        # maxPixels=1e+13
                    )

                    task.start()
            except Exception as e:
                print(e)
        

LE07_001058_20221219 4
Collection.loadTable: Collection asset 'projects/earthengine-legacy/assets/projects/imazon-simex/LULC/SAMPLES/COLLECTION7/TRAINED_BY_SCENE/2022/LE07_001058_20221219_1' not found.
LE07_001064_20220903 4
Collection.loadTable: Collection asset 'projects/earthengine-legacy/assets/projects/imazon-simex/LULC/SAMPLES/COLLECTION7/TRAINED_BY_SCENE/2022/LE07_001064_20220903_1' not found.
LE07_001065_20220903 4
Collection.loadTable: Collection asset 'projects/earthengine-legacy/assets/projects/imazon-simex/LULC/SAMPLES/COLLECTION7/TRAINED_BY_SCENE/2022/LE07_001065_20220903_1' not found.
LE07_001066_20220903 4
Collection.loadTable: Collection asset 'projects/earthengine-legacy/assets/projects/imazon-simex/LULC/SAMPLES/COLLECTION7/TRAINED_BY_SCENE/2022/LE07_001066_20220903_1' not found.
LE07_002057_20220222 4
Collection.loadTable: Collection asset 'projects/earthengine-legacy/assets/projects/imazon-simex/LULC/SAMPLES/COLLECTION7/TRAINED_BY_SCENE/2022/LE07_002057_20220222_1' n